### This is the Notebook for In-Class Coding 2 - Logic Design - Full Adder

In this lecture, we will learn
<ol>
    <li>Develop and test a 1-bit Full Adder</li>
    <li>Use Recursion to develop a 4-bit Full Adder with connected gates</li>
</ol>
<br />
First, we will run the code blocks we developed in Mondays's lecture:

In [1]:
# You will do this library import with every PyRTL assignment
from pyrtl import *
import pyrtl

In [2]:
def two_input_and( a_input, b_input ):
    
    # Create the wire out and put a & b on that 
    y_output = a_input & b_input

    # Use assert to ensure that the signals are one bit
    return y_output

In [3]:
def two_input_or( a_input, b_input ):
    
    # Create the wire out and put a | b on that 
    y_output = a_input | b_input

    # Use assert to ensure that the signals are one bit
    return y_output

In [4]:
def two_input_xor( a_input, b_input ):
    
    # Create the wire out and put a ^ b on that 
    y_output = a_input ^ b_input

    # Use assert to ensure that the signals are one bit
    return y_output

## Classic Advanced Circuit: Adder

Now we will build the full adder that we developed together in class on Mnday

Recall that the logic <b>equations</b> we developed were as follows:
<ol>
    <li><code>Sum = a ^ b ^ Cin</code></li>
    <li><code>Cout = (a & b) | (Cin & (a ^ b))</code></li>
</ol>

The <b>logic diagram</b> and <b>truth table</b> are presented below:

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Full-adder_logic_diagram.svg/400px-Full-adder_logic_diagram.svg.png"></center> <br />

| A | B | Cin | Sum | Cout |
|---|---|---|---|---|
|0|0|0|0|0|
|0|0|1|1|0|
|0|1|0|1|0|
|0|1|1|0|1|
|1|0|0|1|0|
|1|0|1|0|1|
|1|1|0|0|1|
|1|1|1|1|1|

In [5]:
def sum( a, b, c_in ):
    
    # In-Class: Return the sum using previously design cells
    # Could also simply do a ^ b ^ cin
    return two_input_xor( two_input_xor(a, b), c_in )

In [6]:
def carry_out( a, b, c_in ):
    
    # In-Class: Return the carry out
    return (a & b) | ( c_in & (a ^ b) )

In [7]:
def full_adder( a, b, c_in ):
    
    # In-Class: Calculate the sum
    output_sum = sum( a, b, c_in )

    # In-Class: Calculate the carry out
    output_c_out = carry_out( a, b, c_in )
    
    # Return both
    return output_sum, output_c_out

In [8]:
def full_adder_simulate():
    
    # Step 1 - Reset the working block
    pyrtl.reset_working_block()
    
    # In-Class: Step 2 - Create the input and ouput wires
    a, b, c_in = pyrtl.Input(1, 'a'), pyrtl.Input(1, 'b'), pyrtl.Input(1, 'c_in')
    output_sum = pyrtl.Output(1, 'output_sum')
    output_c_out = pyrtl.Output(1, 'output_c_out')
    
    # In-Class: Step 3-a - Save to an intermediate value using the three_input_and_or function
    inter_sum, inter_c_out = full_adder( a, b, c_in )
    
    # In-Class: Step 3-b Assign to a wire using <<=
    output_sum <<= inter_sum
    output_c_out <<= inter_c_out
    
    # Step 4 - - Simulate the design
    sim = pyrtl.Simulation()
    
    # Step 5 - Create lists for the inputs
    a_inputs =    [0,0,0,0,1,1,1,1]
    b_inputs =    [0,0,1,1,0,0,1,1]
    c_in_inputs = [0,1,0,1,0,1,0,1]
    
    # Step 6 - Loop through and simuluate
    for value in range(0, len(a_inputs)):

        sim.step({
            'a' : a_inputs[value],
            'b' : b_inputs[value],
            'c_in' : c_in_inputs[value] 
        })
    
    # Render the trace
    sim.tracer.render_trace()

In [9]:
# Call the function
full_adder_simulate()

<IPython.core.display.Javascript object>

### Combining Concepts: Using Recursion to Create a Full Adder

When we create a circuit in PyRTL, we cannot use loop to create the signals, only the input signals. But if there is repetition, we can use <b>recursion</b>!

In [10]:
def ripple_carry_adder( a, b, c_in ):
    
    # Base Case: When a is a length of 1, we put the first full adder
    if len(a) == 1:
        sum_result, c_out = full_adder(a, b, c_in )
    
    # Recursive Case
    else:

        # Get the "least significant" bit and the rippecarry to the next full adder
        ls_bit, ripplecarry = full_adder(a[0], b[0], c_in)
        
        # Recursively call the remaining bits to get the "most significant" bits and the current carry out
        ms_bits, c_out = ripple_carry_adder( a[1:], b[1:], ripplecarry )
        
        # Use pyrtl.concat Concatenate the results on the recursive call back up
        sum_result = pyrtl.concat(ms_bits, ls_bit)

    # Return the results                                      
    return sum_result, c_out

In [11]:
def ripple_carry_sim( ):
    
    # Step 1 - Reset the working block
    pyrtl.reset_working_block()
    
    # In-Class: Step 2 - Create the input and ouput wires for a 4-bit adder
    a, b, c_in = pyrtl.Input(4, 'a'), pyrtl.Input(4, 'b'), pyrtl.Input(1, 'c_in')
    output_sum = pyrtl.Output(4, 'output_sum')
    output_c_out = pyrtl.Output(1, 'output_c_out')    
    
    
    # In-Class:Step 3-a - Save to an intermediate value using the three_input_and_or function
    inter_sum, inter_c_out = ripple_carry_adder(a, b, c_in)
    
    # In-Class: Step 3-b Assign to a bus wire using <<=
    output_sum <<= inter_sum
    output_c_out <<= inter_c_out
    
    # Step 4 - Simulate the design
    sim = pyrtl.Simulation()
    
    # Step 5 - Create lists for the inputs
    a_inputs = [ 0, 3, 10, 2, 15, 3, 14 ]
    b_inputs = [ 4, 6, 6, 11, 15, 12, 1 ]
    c_in_inputs = [0, 0, 0, 0, 0, 0, 0]

    # Step 6 - Loop through and simuluate
    for value in range( 0, len(a_inputs) ):

        sim.step({
            'a' : a_inputs[value],
            'b' : b_inputs[value],
            'c_in' : c_in_inputs[value] 
        })
    
    # Render the trace
    sim.tracer.render_trace()

In [12]:
ripple_carry_sim( )

<IPython.core.display.Javascript object>